In [66]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

In [67]:
class AgentState(TypedDict):
  num1: int
  num2: int
  op1: str
  num3: int
  num4: int
  op2: str
  final_rslt1: int
  final_rslt2: int


In [78]:
def first_add_op(state: AgentState) -> AgentState:
  """
  This is the first adder func between 2 nums
  """
  state["final_rslt1"] = state["num1"] + state["num2"]
  return state

def first_sub_op(state: AgentState) -> AgentState:
  """
  this is the first subtraction func between 2 nums
  """
  state["final_rslt1"] = state["num1"] - state["num2"]
  return state

def decide_first_op(state: AgentState) -> AgentState:
  """
  This routes the first arithmetic operation
  """
  if state["op1"] == '+':
    return "addition_op1"

  elif state["op1"] == '-':
    return "subtraction_op1"

def second_add_op(state: AgentState) -> AgentState:
  """
  This is the first adder func between 2 nums
  """
  state["final_rslt2"] = state["num3"] + state["num4"]
  return state

def second_sub_op(state: AgentState) -> AgentState:
  """
  this is the first subtraction func between 2 nums
  """
  state["final_rslt2"] = state["num3"] - state["num4"]

def decide_second_op(state: AgentState) -> AgentState:
  """
  This routes the first arithmetic operation
  """
  if state["op2"] == '+':
    return "addition_op2"

  elif state["op2"] == '-':
    return "subtraction_op2"

In [79]:
graph = StateGraph(AgentState)

graph.add_node("add1", first_add_op)
graph.add_node("sub1", first_sub_op)
graph.add_node("add2", second_add_op)
graph.add_node("sub2", second_sub_op)
graph.add_node("router1", lambda state: state)
graph.add_node("router2", lambda state: state)

# First Approach: using `set_entry_point`, `set_finish_point`
# graph.set_entry_point("router1")
# graph.add_conditional_edges(
#     "router1",
#     decide_first_op,
#     {
#         "addition_op1": "add1",
#         "subtraction_op1": "sub1"
#     }
# )
# graph.add_edge("add1", "router2")
# graph.add_edge("sub1", "router2")
# graph.add_conditional_edges(
#     "router2",
#     decide_second_op,
#     {
#         "addition_op2": "add2",
#         "subtraction_op2": "sub2"
#     }
# )
# graph.set_finish_point("add2")
# graph.set_finish_point("sub2")

# Second Approach: using `START`, `END`

graph.add_edge(START, "router1")
graph.add_conditional_edges(
    "router1",
    decide_first_op,
    {
        "addition_op1": "add1",
        "subtraction_op1": "sub1"
    }
)
graph.add_edge("add1", "router2")
graph.add_edge("sub1", "router2")
graph.add_conditional_edges(
  "router2",
  decide_second_op,
  {
      "addition_op2": "add2",
      "subtraction_op2": "sub2"
  }
)
graph.add_edge("add2", END)
graph.add_edge("sub2", END)

app = graph.compile()

In [80]:
# from IPython.display import Image, display
# # dag = app.get_graph()
# display(Image(app.get_graph().draw_mermaid_png()))

In [81]:
rslt = app.invoke({"num1":3, "num2":1, "op1":'-',
"num3":5, "num4": 4, "op2": '+'})
rslt["final_rslt1"], rslt["final_rslt2"]
# print(rslt)

(2, 9)